In [1]:
import requests
import pandas as pd
import re

from datetime import datetime
from bs4 import BeautifulSoup

# func

In [2]:
# get web content
def fetch_url(url):
    cookies = {
    'fastivalName_Mall_20240402': 'closeday_fastivalName_Mall_20240402',
    'bottomADName_20240402': 'closeday_bottomADName_20240402',
    '_ga': 'GA1.2.1503886441.1586358579',
    'G_ENABLED_IDPS': 'google',
    '__gsas': 'ID=8cb022d19004ab5d:T=1695296450:RT=1695296450:S=ALNI_MZMEwrGadA-C1w3uSGLQNid8VzhCA',
    '__gpi': 'UID=00000d91fda9a60c:T=1694523716:RT=1695298093:S=ALNI_Mamjv44DbNeXB2ZMvKkUthCe1HHOA',
    '_fbp': 'fb.1.1709200912916.558103157',
    '__gads': 'ID=d33936820c2bf19e-2293b130d9e300da:T=1694523716:RT=1709814441:S=ALNI_MbdJVWtbhNsdx3WCETHImKMSTH1jA',
    '__eoi': 'ID=a1c808fead30c63a:T=1709200913:RT=1709814441:S=AA-AfjahzsuZaasJmq6DAQKXiJnq',
    'ASP.NET_SessionId': '5y3xtv104w5pzl1iqh3fbln4',
    '_gcl_au': '1.1.761990035.1719583062',
    '_gid': 'GA1.2.748811600.1720146354',
    'g_state': '{"i_l":0}',
    'NickName': 'w25225575',
    'MemberNo': '109104',
    'Email': 'w25225575@gmail.com',
    '_gat': '1',
    '_ga_S0YRRCXLNT': 'GS1.2.1720235863.14.1.1720238222.60.0.0'
    }
    response = requests.get(url, cookies=cookies)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

def try_float(s):
    try:
        if '%' in s:
            s = s.replace('%', '')
        return float(s.replace(',', ''))
    except:
        return s

def get_bno(s):
    match = re.search(r'bno=([^&]+)', s)
    if match:
        return match.group(1)
    else:
        return s

def scrape_today_broker_data(stock_no):
    main_url = f"https://histock.tw/stock/branch.aspx?no={stock_no}"
    main_soup = fetch_url(main_url)
    data = []
    table = main_soup.find_all("table", {"class": "tb-stock tbChip tbHide"})
    if table:
        trs = table[0].find_all('tr')[1:]
        for tr in trs:
            tds = tr.find_all('td')
            temp = []
            for td in tds:
                if td.findChild('a'):
                    td_a = td.findChild('a')
                    temp.append(td_a['href'])
                temp.append(try_float(td.string))
            # 切割清單
            data.extend([temp[i:i+6] for i in range(0, len(temp), 6)])
                            
    df = pd.DataFrame(data)
    df.rename(columns={0: 'url', 1: 'broker_name', 2: 'buy', 3: 'sell', 4: 'over', 5: 'price'}, inplace=True)
    df.fillna(0, inplace=True)
    bno_df = df['url'].apply(get_bno)
    df.insert(loc=1, column='bno', value=bno_df)
    return df

def scrape_brokertrace(stock_no, bno):
    main_url = f"https://histock.tw/stock/brokertrace.aspx?bno={bno}&no={stock_no}"
    main_soup = fetch_url(main_url)
    data = []
    table = main_soup.find_all("table", {"class": "tbTable tb-stock tbChip"})
    if table:
        trs = table[0].find_all('tr')[1:]
        for tr in trs:
            tds = tr.find_all('td')
            temp = []
            for td in tds:                
                temp.append(try_float(td.string))
            data.extend([temp])
    
    df = pd.DataFrame(data)
    df.rename(columns={0: 'date', 1: 'buy', 2: 'buy_price', 3: 'sell', 4: 'sell_prices', 5: 'price', 6: 'over', 7: '60day', 8: 'deal_rate', 9: 'info'}, inplace=True)
    df.fillna(0, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    return df

def scrape_broker_profit(stock_no, day=None):
    url = f"https://histock.tw/stock/mainprofit.aspx?no={stock_no}"
    if day : 
        url = url + f"&day={day}"

    main_soup = fetch_url(url)
    data = []
    table = main_soup.find_all("table", {"class": "tbTable tb-stock tbChip"})
    if table:
        trs = table[0].find_all('tr')[1:]
        for tr in trs:
            tds = tr.find_all('td')
            tds = tds[1:]
            temp = []
            for td in tds:                
                if td.findChild('a'):
                    td_a = td.findChild('a')
                    temp.append(td_a.string)
                    temp.append(td_a['href'])
                else:
                    temp.append(try_float(td.string))
            data.extend([temp])
    
    df = pd.DataFrame(data)
    df.rename(columns={0: 'bro_n', 1: 'url', 2: 'profit_1', 3: 'profit_2', 4: 'realized', 5: 'unrealized', 6: 'over', 7: 'buy', 8: 'sell', 9: 'avg_price', 10: 'avg_buy', 11: 'avg_sell', 12: 'price'}, inplace=True)
    df.fillna(0, inplace=True)
    bno_df = df['url'].apply(get_bno)
    df.insert(loc=1, column='bno', value=bno_df)
    return df


In [17]:
stock_no = '3131'
bno = '5920'

In [63]:
borker_profit = scrape_broker_profit(stock_no)

In [64]:
profit_df = borker_profit.copy()
profit_df = profit_df.sort_values(by=['profit_1', 'profit_2'], ascending=False)
profit_df

,bro_n,bno,url,profit_1,profit_2,realized,unrealized,over,buy,sell,avg_price,avg_buy,avg_sell,price
0,永豐金-埔里,9A79,/stock/brokertrace.aspx?bno=9A79&no=3131,10.00,149.0,149.0,0.0,0.0,1.0,1.0,1490.1,1416.0,1565.0,1565.0
1,台灣企銀-台中,1111,/stock/brokertrace.aspx?bno=1111&no=3131,7.93,115.0,0.0,115.0,0.0,1.0,0.0,1450.6,1450.0,1565.0,1565.0
2,元大-羅東,982A,/stock/brokertrace.aspx?bno=982A&no=3131,6.32,575.0,575.0,0.0,0.0,6.0,6.0,1517.1,1469.2,1565.0,1565.0
3,新光-桃園,8565,/stock/brokertrace.aspx?bno=8565&no=3131,5.69,84.7,84.7,0.0,0.0,1.0,1.0,1488.1,1445.5,1530.2,1565.0
4,元大-鳳山,983Z,/stock/brokertrace.aspx?bno=983Z&no=3131,5.06,75.0,75.0,0.0,0.0,1.0,1.0,1482.5,1445.0,1520.0,1565.0
5,美好-蘆洲,5261,/stock/brokertrace.aspx?bno=5261&no=3131,5.03,75.0,0.0,75.0,0.0,1.0,0.0,1490.4,1490.0,1525.0,1565.0
6,凱基-岡山,9209,/stock/brokertrace.aspx?bno=9209&no=3131,4.83,70.0,70.0,0.0,0.0,1.0,1.0,1450.0,1415.0,1485.0,1565.0
7,統一-三重,585J,/stock/brokertrace.aspx?bno=585J&no=3131,4.71,70.0,70.0,0.0,0.0,1.0,1.0,1485.0,1450.0,1520.0,1565.0
8,元大-鳳中,985C,/stock/brokertrace.aspx?bno=985C&no=3131,4.69,349.9,349.9,0.0,0.0,5.0,5.0,1491.0,1456.0,1526.0,1565.0
9,國票-台東,7791,/stock/brokertrace.aspx?bno=7791&no=3131,4.59,70.0,70.0,0.0,0.0,1.0,1.0,1525.0,1490.0,1560.0,1565.0


In [3]:
broker_trace = scrape_brokertrace('3131', '9A79')
broker_trace

,date,buy,buy_price,sell,sell_prices,price,over,60day,deal_rate,info
0,2024-07-08,3.0,1561.67,3.0,1465.00,1465.0,0.0,807.0,0.00,\n
1,2024-07-05,1.0,1416.04,1.0,1565.00,1565.0,0.0,802.0,0.00,\n
2,2024-07-04,1.0,1425.00,1.0,1425.00,1425.0,0.0,806.0,0.00,\n
3,2024-07-03,0.0,0.00,1.0,1375.00,1375.0,-1.0,819.0,-0.12,\n
4,2024-07-01,1.0,1415.00,1.0,1395.29,1415.0,0.0,837.0,0.00,\n
...,...,...,...,...,...,...,...,...,...,...
361,2018-09-13,1.0,115.50,0.0,0.00,116.0,1.0,52.0,1.90,\n
362,2018-08-13,1.0,128.00,0.0,0.00,127.5,1.0,69.0,1.44,\n
363,2018-07-05,1.0,141.50,0.0,0.00,140.5,1.0,69.0,1.44,\n
364,2018-03-19,0.0,0.00,1.0,175.50,176.0,-1.0,78.0,-1.28,\n


In [4]:
x = []

for i in range(2, len(broker_trace)):
    x.append(broker_trace.iloc[i-2:i])